#Sistema de Predição de Cáries


##Importando bibliotecas e datase

In [ ]:
#Bibliotecas utilizadas
#mudar diretorio
import os
#manipulacao de dados
import pandas as pd
import numpy as np
#graficos
import matplotlib.pyplot as plt
import seaborn as sns
#metricas
from sklearn.metrics import classification_report
#classificadores selecionados
#from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#tunning e split
from sklearn.model_selection import train_test_split, GridSearchCV
#validacao cruzada
from sklearn.model_selection import cross_val_score, KFold
#padronizacao
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline
#selecao de features
from sklearn.linear_model import Lasso
#tratamento de dados faltantes
from sklearn.impute import KNNImputer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#mudar diretorio
os.chdir("/content/drive/MyDrive/Colab Notebooks/ML_SAUDE/TEMA")
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/ML_SAUDE/TEMA'

#Criação do DATAFRAME

In [ ]:
#importar datasets
data_xpts = os.listdir('Dataset/NHANES_2013_2014/caries/') #datasets utilizados no artigo
data_xpts.remove('OHXDEN_H.XPT')
data_xpts.remove('OHXREF_H.XPT')
data_xpts


['DEMO_H.XPT', 'DR1TOT_H.XPT']

In [ ]:
data_y = pd.read_sas('Dataset/NHANES_2013_2014/caries/OHXDEN_H.XPT'); #y
data_y=data_y.loc[data_y.OHDEXSTS == 1] #excluido pessoas que não fizeram o exame completo
data_y.set_index('SEQN',inplace = True)
data_y = data_y.iloc[:,35:63]#colunas de interesse 35 a 62
data_y = data_y.applymap(lambda x: x.decode('UTF-8'))
data_y.reset_index(inplace=True)
data_y.rename(columns={"index": "SEQN"},inplace=True)
data_y

<ipython-input-5-b93bcc6976be>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_y = pd.read_sas('Dataset/NHANES_2013_2014/caries/OHXDEN_H.XPT'); #y
<ipython-input-5-b93bcc6976be>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_y = pd.read_sas('Dataset/NHANES_2013_2014/caries/OHXDEN_H.XPT'); #y
<ipython-input-5-b93bcc6976be>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

,SEQN,OHX02CTC,OHX03CTC,OHX04CTC,OHX05CTC,OHX06CTC,OHX07CTC,OHX08CTC,OHX09CTC,OHX10CTC,...,OHX22CTC,OHX23CTC,OHX24CTC,OHX25CTC,OHX26CTC,OHX27CTC,OHX28CTC,OHX29CTC,OHX30CTC,OHX31CTC
0,73557.0,P,P,P,P,P,P,P,P,P,...,P,P,P,P,P,P,P,P,P,P
1,73558.0,Z,J,S,S,S,S,S,S,S,...,S,S,S,S,S,S,Z,S,E,E
2,73559.0,Z,Z,Z,S,S,S,S,Z,S,...,S,Z,S,S,S,S,S,Z,Z,E
3,73560.0,U,S,D,K,U,S,S,S,S,...,D,S,S,S,S,U,D,D,S,U
4,73561.0,E,Z,E,Z,Z,Z,R,R,Z,...,S,Z,R,Z,S,Z,E,E,E,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,S,S,S,M,S,S,S,S,S,...,S,S,S,S,S,S,M,S,Z,S
8629,83728.0,U,U,U,D,D,D,D,D,D,...,D,D,D,D,D,D,D,U,U,U
8630,83729.0,Z,Z,E,Z,S,S,S,S,S,...,S,S,S,S,S,S,S,S,E,Z
8631,83730.0,U,S,K,K,D,D,S,S,D,...,D,S,S,S,D,D,K,K,S,U


In [ ]:
data_y['carie'] = 0
caries_check = data_y.columns[1:-1] #colunas a serem checadas
n_reg = data_y.shape[0] #numero de registros
caries_l = ["E","P","R"] #pacientes que perderam os dentes
#caries_l = ["Z"] #pacientes com carie
for col in caries_check:
  for i in range(n_reg):
    if data_y[col][i] in caries_l:
      data_y.carie[i] = 1
data_y

<ipython-input-6-dec85f60787e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_y.carie[i] = 1


,SEQN,OHX02CTC,OHX03CTC,OHX04CTC,OHX05CTC,OHX06CTC,OHX07CTC,OHX08CTC,OHX09CTC,OHX10CTC,...,OHX23CTC,OHX24CTC,OHX25CTC,OHX26CTC,OHX27CTC,OHX28CTC,OHX29CTC,OHX30CTC,OHX31CTC,carie
0,73557.0,P,P,P,P,P,P,P,P,P,...,P,P,P,P,P,P,P,P,P,1
1,73558.0,Z,J,S,S,S,S,S,S,S,...,S,S,S,S,S,Z,S,E,E,1
2,73559.0,Z,Z,Z,S,S,S,S,Z,S,...,Z,S,S,S,S,S,Z,Z,E,1
3,73560.0,U,S,D,K,U,S,S,S,S,...,S,S,S,S,U,D,D,S,U,0
4,73561.0,E,Z,E,Z,Z,Z,R,R,Z,...,Z,R,Z,S,Z,E,E,E,E,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,S,S,S,M,S,S,S,S,S,...,S,S,S,S,S,M,S,Z,S,0
8629,83728.0,U,U,U,D,D,D,D,D,D,...,D,D,D,D,D,D,U,U,U,0
8630,83729.0,Z,Z,E,Z,S,S,S,S,S,...,S,S,S,S,S,S,S,E,Z,1
8631,83730.0,U,S,K,K,D,D,S,S,D,...,S,S,S,D,D,K,K,S,U,0


In [ ]:
data_y.carie.value_counts()  #contagem de valores

0    5728
1    2905
Name: carie, dtype: int64

In [ ]:
data_carie = data_y.drop(columns=caries_check) #excluir colunas ja transformadas
data_carie

,SEQN,carie
0,73557.0,1
1,73558.0,1
2,73559.0,1
3,73560.0,0
4,73561.0,1
...,...,...
8628,83727.0,0
8629,83728.0,0
8630,83729.0,1
8631,83730.0,0


In [ ]:
df = data_carie.copy()
for files in data_xpts: #merging
  df1 = pd.read_sas('Dataset/NHANES_2013_2014/caries/'+ files);
  df = pd.merge(df, df1, how = 'left', on = 'SEQN');

<ipython-input-9-50e3d8b1a90f>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1 = pd.read_sas('Dataset/NHANES_2013_2014/caries/'+ files);
<ipython-input-9-50e3d8b1a90f>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1 = pd.read_sas('Dataset/NHANES_2013_2014/caries/'+ files);
<ipython-input-9-50e3d8b1a90f>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

In [ ]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)
df

,SEQN,carie,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,...,DRD370QQ,DRD370R,DRD370RQ,DRD370S,DRD370SQ,DRD370T,DRD370TQ,DRD370U,DRD370UQ,DRD370V
0,73557.0,1,8.0,2.0,1.0,69.0,NaN,4.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73558.0,1,8.0,2.0,1.0,54.0,NaN,3.0,3.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
2,73559.0,1,8.0,2.0,1.0,72.0,NaN,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73560.0,0,8.0,2.0,1.0,9.0,NaN,3.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73561.0,1,8.0,2.0,2.0,73.0,NaN,3.0,3.0,1.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,0,8.0,2.0,1.0,26.0,NaN,2.0,2.0,2.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
8629,83728.0,0,8.0,2.0,2.0,2.0,24.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8630,83729.0,1,8.0,2.0,2.0,42.0,NaN,4.0,4.0,2.0,...,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,2.0
8631,83730.0,0,8.0,2.0,1.0,7.0,NaN,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for col in df.columns:
  if ((df[col].isna().sum()) >= (int(df.shape[0]/2))):
    df.drop(columns=col,inplace=True)

df

,SEQN,carie,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,...,DRD370M,DRD370N,DRD370O,DRD370P,DRD370Q,DRD370R,DRD370S,DRD370T,DRD370U,DRD370V
0,73557.0,1,8.0,2.0,1.0,69.0,4.0,4.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73558.0,1,8.0,2.0,1.0,54.0,3.0,3.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,73559.0,1,8.0,2.0,1.0,72.0,3.0,3.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73560.0,0,8.0,2.0,1.0,9.0,3.0,3.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73561.0,1,8.0,2.0,2.0,73.0,3.0,3.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,0,8.0,2.0,1.0,26.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8629,83728.0,0,8.0,2.0,2.0,2.0,1.0,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8630,83729.0,1,8.0,2.0,2.0,42.0,4.0,4.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8631,83730.0,0,8.0,2.0,1.0,7.0,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.isna().sum()

SEQN           0
carie          0
SDDSRVYR       0
RIDSTATR       0
RIAGENDR       0
            ... 
DRD370R     3929
DRD370S     3929
DRD370T     3929
DRD370U     3929
DRD370V     3930
Length: 153, dtype: int64

#Pré-processamento


##Remoção de Outliers e OneHotEncoder

In [ ]:
#Removendo outliers
#DBD100 - 7,9
#DRQSPREP - 9
#DR1STY - 9
#DRQSDIET - 9
#DR1_300 - 7,9
#DRD340 - 7,9
#DRD360 - 7,9
#DMQMILIZ  - 7,9
#DMDBORN4 - 77,99
#DMDCITZN -7,9
outliers_7_9 = ['DBD100','DR1_300','DRD340','DRD360','DMQMILIZ','DMDCITZN']
outliers_9 = ['DRQSPREP','DR1STY','DRQSDIET']
for i in range(df.shape[0]):
  for col in outliers_7_9:
    if ((df[col][i] >= 7)):
      df[col][i] = np.nan
  for col in outliers_9:
    if ((df[col][i] == 9)):
      df[col][i] = np.nan
  if (df['DMDBORN4'][i] >= 77):
    df['DMDBORN4'][i] = np.nan
df

<ipython-input-13-25e80c8b2efb>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][i] = np.nan
<ipython-input-13-25e80c8b2efb>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][i] = np.nan
<ipython-input-13-25e80c8b2efb>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DMDBORN4'][i] = np.nan


,SEQN,carie,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,...,DRD370M,DRD370N,DRD370O,DRD370P,DRD370Q,DRD370R,DRD370S,DRD370T,DRD370U,DRD370V
0,73557.0,1,8.0,2.0,1.0,69.0,4.0,4.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,73558.0,1,8.0,2.0,1.0,54.0,3.0,3.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,73559.0,1,8.0,2.0,1.0,72.0,3.0,3.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73560.0,0,8.0,2.0,1.0,9.0,3.0,3.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73561.0,1,8.0,2.0,2.0,73.0,3.0,3.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,0,8.0,2.0,1.0,26.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8629,83728.0,0,8.0,2.0,2.0,2.0,1.0,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8630,83729.0,1,8.0,2.0,2.0,42.0,4.0,4.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8631,83730.0,0,8.0,2.0,1.0,7.0,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#ONE HOT ENCODER
dummies=['DR1DRSTZ','DR1DAY','DR1LANG','DR1MNRSP','DR1HELPD','DBQ095Z','DR1TWS',
         'RIDRETH1','RIDRETH3','DMDEDUC2','DMDMARTL','AIALANGA','DMDHRBR4',
         'DMDHREDU','DMDHRMAR','DMDHSEDU','INDHHIN2','INDFMIN2']
df_dummy = df[dummies].astype(str)
df_dummy2 = pd.get_dummies(df_dummy, dtype=float,dummy_na=False)
df_dummy2

,DR1DRSTZ_1.0,DR1DRSTZ_2.0,DR1DRSTZ_4.0,DR1DRSTZ_5.0,DR1DAY_1.0,DR1DAY_2.0,DR1DAY_3.0,DR1DAY_4.0,DR1DAY_5.0,DR1DAY_6.0,...,INDFMIN2_3.0,INDFMIN2_4.0,INDFMIN2_5.0,INDFMIN2_6.0,INDFMIN2_7.0,INDFMIN2_77.0,INDFMIN2_8.0,INDFMIN2_9.0,INDFMIN2_99.0,INDFMIN2_nan
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8629,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8630,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8631,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df=pd.concat([df,df_dummy2],axis=1)
df

,SEQN,carie,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,...,INDFMIN2_3.0,INDFMIN2_4.0,INDFMIN2_5.0,INDFMIN2_6.0,INDFMIN2_7.0,INDFMIN2_77.0,INDFMIN2_8.0,INDFMIN2_9.0,INDFMIN2_99.0,INDFMIN2_nan
0,73557.0,1,8.0,2.0,1.0,69.0,4.0,4.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,73558.0,1,8.0,2.0,1.0,54.0,3.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,73559.0,1,8.0,2.0,1.0,72.0,3.0,3.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,73560.0,0,8.0,2.0,1.0,9.0,3.0,3.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,73561.0,1,8.0,2.0,2.0,73.0,3.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,0,8.0,2.0,1.0,26.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8629,83728.0,0,8.0,2.0,2.0,2.0,1.0,1.0,2.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8630,83729.0,1,8.0,2.0,2.0,42.0,4.0,4.0,2.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8631,83730.0,0,8.0,2.0,1.0,7.0,2.0,2.0,1.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df

,SEQN,carie,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,...,INDFMIN2_3.0,INDFMIN2_4.0,INDFMIN2_5.0,INDFMIN2_6.0,INDFMIN2_7.0,INDFMIN2_77.0,INDFMIN2_8.0,INDFMIN2_9.0,INDFMIN2_99.0,INDFMIN2_nan
0,73557.0,1,8.0,2.0,1.0,69.0,4.0,4.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,73558.0,1,8.0,2.0,1.0,54.0,3.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,73559.0,1,8.0,2.0,1.0,72.0,3.0,3.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,73560.0,0,8.0,2.0,1.0,9.0,3.0,3.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,73561.0,1,8.0,2.0,2.0,73.0,3.0,3.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8628,83727.0,0,8.0,2.0,1.0,26.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8629,83728.0,0,8.0,2.0,2.0,2.0,1.0,1.0,2.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8630,83729.0,1,8.0,2.0,2.0,42.0,4.0,4.0,2.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8631,83730.0,0,8.0,2.0,1.0,7.0,2.0,2.0,1.0,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


##Normalização

In [ ]:
df_c = df.copy()
colunas_df = df.columns
df.drop(columns=['SEQN'], inplace=True)

y = df[['carie']]
X = df.drop(columns=['carie'])
X_c = X.copy()
colunas_X = X.columns

# define standard scaler
scaler = StandardScaler()

# transform data
X = scaler.fit_transform(X)

X = pd.DataFrame(X,columns = colunas_X )

#import scipy
#from scipy import stats
#for col in colunas_x:
#  X[col] = stats.zscore(X[col])



In [ ]:
#divisão da base
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.3, random_state=0)
X_train

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,DMDBORN4,DMDCITZN,...,INDFMIN2_3.0,INDFMIN2_4.0,INDFMIN2_5.0,INDFMIN2_6.0,INDFMIN2_7.0,INDFMIN2_77.0,INDFMIN2_8.0,INDFMIN2_9.0,INDFMIN2_99.0,INDFMIN2_nan
3780,0.0,0.0,-1.001507,0.078503,-0.072416,-0.179431,0.981066,0.301172,-0.480544,-0.32286,...,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527
3887,0.0,0.0,0.998495,1.099997,0.718930,0.439933,-1.019299,0.301172,-0.480544,-0.32286,...,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527
2015,0.0,0.0,-1.001507,-0.985553,-0.072416,-0.179431,0.981066,NaN,-0.480544,-0.32286,...,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527
1742,0.0,0.0,0.998495,0.887186,0.718930,0.439933,-1.019299,0.301172,-0.480544,-0.32286,...,-0.266132,-0.271581,-0.308675,2.733502,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527
7196,0.0,0.0,-1.001507,-0.985553,0.718930,0.439933,0.981066,NaN,-0.480544,-0.32286,...,-0.266132,-0.271581,3.239651,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,0.0,0.0,0.998495,-1.198364,-1.655107,-1.418161,0.981066,NaN,-0.480544,-0.32286,...,-0.266132,-0.271581,3.239651,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527
2333,0.0,0.0,0.998495,-0.942990,0.718930,0.439933,-1.019299,NaN,-0.480544,-0.32286,...,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,3.533987,-0.230908,-0.082953,-0.098527
8369,0.0,0.0,0.998495,1.099997,0.718930,0.439933,-1.019299,0.301172,-0.480544,-0.32286,...,-0.266132,-0.271581,-0.308675,2.733502,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527
3593,0.0,0.0,0.998495,0.121065,0.718930,0.439933,-1.019299,0.301172,-0.480544,-0.32286,...,-0.266132,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,3.533987,-0.230908,-0.082953,-0.098527


##Tratamento de Dados Faltantes

In [ ]:
df_train =  pd.merge(X_train, y_train, left_index=True, right_index=True);
df_train.head()
df_train_c = df_train.copy()

In [ ]:
imputer = KNNImputer(n_neighbors=3)
df_train_knn = imputer.fit_transform(df_train)
df_train_knn


array([[ 0.        ,  0.        , -1.00150699, ..., -0.08295341,
        -0.09852717,  0.        ],
       [ 0.        ,  0.        ,  0.99849528, ..., -0.08295341,
        -0.09852717,  0.        ],
       [ 0.        ,  0.        , -1.00150699, ..., -0.08295341,
        -0.09852717,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.99849528, ..., -0.08295341,
        -0.09852717,  1.        ],
       [ 0.        ,  0.        ,  0.99849528, ..., -0.08295341,
        -0.09852717,  0.        ],
       [ 0.        ,  0.        ,  0.99849528, ..., -0.08295341,
        -0.09852717,  0.        ]])

In [ ]:
#substituindo valores tratados
for i in range(df_train.shape[0]):
  df_train.iloc[i,:] = df_train_knn[i]

In [ ]:
df_train.head()

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,DMDBORN4,DMDCITZN,...,INDFMIN2_4.0,INDFMIN2_5.0,INDFMIN2_6.0,INDFMIN2_7.0,INDFMIN2_77.0,INDFMIN2_8.0,INDFMIN2_9.0,INDFMIN2_99.0,INDFMIN2_nan,carie
3780,0.0,0.0,-1.001507,0.078503,-0.072416,-0.179431,0.981066,0.301172,-0.480544,-0.32286,...,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,0
3887,0.0,0.0,0.998495,1.099997,0.718930,0.439933,-1.019299,0.301172,-0.480544,-0.32286,...,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,0
2015,0.0,0.0,-1.001507,-0.985553,-0.072416,-0.179431,0.981066,0.301172,-0.480544,-0.32286,...,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,0
1742,0.0,0.0,0.998495,0.887186,0.718930,0.439933,-1.019299,0.301172,-0.480544,-0.32286,...,-0.271581,-0.308675,2.733502,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,1
7196,0.0,0.0,-1.001507,-0.985553,0.718930,0.439933,0.981066,0.301172,-0.480544,-0.32286,...,-0.271581,3.239651,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,0


In [ ]:
y_train = df_train[['carie']]
X_train =  df_train.drop(columns=['carie'])
y_train.value_counts()


carie
0        4010
1        2033
dtype: int64

In [ ]:
df_test = pd.merge(X_test, y_test, left_index=True, right_index=True)
df_test

,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,DMQMILIZ,DMDBORN4,DMDCITZN,...,INDFMIN2_4.0,INDFMIN2_5.0,INDFMIN2_6.0,INDFMIN2_7.0,INDFMIN2_77.0,INDFMIN2_8.0,INDFMIN2_9.0,INDFMIN2_99.0,INDFMIN2_nan,carie
8346,0.0,0.0,0.998495,-0.645055,-1.655107,-1.418161,-1.019299,NaN,-0.480544,-0.32286,...,-0.271581,-0.308675,-0.365831,-0.31204,6.526945,-0.282967,-0.230908,-0.082953,-0.098527,0
4716,0.0,0.0,0.998495,1.312808,-0.863761,-0.798796,0.981066,0.301172,2.080974,-0.32286,...,-0.271581,-0.308675,2.733502,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,1
8553,0.0,0.0,0.998495,-0.687617,0.718930,0.439933,-1.019299,NaN,-0.480544,-0.32286,...,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,3.533987,-0.230908,-0.082953,-0.098527,0
6864,0.0,0.0,-1.001507,0.333877,-0.072416,-0.179431,-1.019299,0.301172,-0.480544,-0.32286,...,-0.271581,-0.308675,2.733502,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,1
980,0.0,0.0,0.998495,-1.283488,0.718930,0.439933,0.981066,NaN,-0.480544,-0.32286,...,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,0.0,0.0,0.998495,-0.857866,0.718930,0.439933,-1.019299,NaN,-0.480544,-0.32286,...,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,0
4336,0.0,0.0,0.998495,1.270246,-0.072416,-0.179431,0.981066,0.301172,-0.480544,-0.32286,...,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,0
3640,0.0,0.0,0.998495,0.121065,-0.072416,-0.179431,-1.019299,0.301172,-0.480544,-0.32286,...,-0.271581,-0.308675,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,1
2835,0.0,0.0,-1.001507,0.121065,-0.072416,-0.179431,0.981066,0.301172,-0.480544,-0.32286,...,-0.271581,3.239651,-0.365831,-0.31204,-0.153211,-0.282967,-0.230908,-0.082953,-0.098527,1


In [ ]:
#df_test.dropna(how='any',inplace=True)
#df_test

##LASSO - feature selection

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
reg.score(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5724280808017125

In [ ]:
coefs_lasso = reg.coef_

In [ ]:
# colunas_X
i = 0;
features_names = []
for coef in coefs_lasso:
  if coef != 0:
    features_names.append(colunas_X[i])
  i += 1
print(len(features_names))
features_names


114


['RIDAGEYR',
 'RIDRETH1',
 'DMQMILIZ',
 'DMDBORN4',
 'DMDCITZN',
 'SIAPROXY',
 'AIALANGA',
 'DMDHHSZA',
 'DMDHHSZB',
 'DMDHRAGE',
 'DMDHREDU',
 'WTMEC2YR',
 'SDMVPSU',
 'INDFMPIR',
 'WTDR2D',
 'DRDINT',
 'DR1DAY',
 'DBQ095Z',
 'DBD100',
 'DR1STY',
 'DRQSDIET',
 'DR1TPROT',
 'DR1TCARB',
 'DR1TSUGR',
 'DR1TFIBE',
 'DR1TATOC',
 'DR1TRET',
 'DR1TLYCO',
 'DR1TLZ',
 'DR1TFF',
 'DR1TVC',
 'DR1TVD',
 'DR1TCOPP',
 'DR1TCAFF',
 'DR1TTHEO',
 'DR1TALCO',
 'DR1TS080',
 'DR1TM201',
 'DR1TM221',
 'DR1TP184',
 'DR1_330Z',
 'DR1TWS',
 'DRD340',
 'DRD360',
 'DRD370A',
 'DRD370B',
 'DRD370F',
 'DRD370G',
 'DRD370H',
 'DRD370I',
 'DRD370J',
 'DRD370M',
 'DRD370N',
 'DRD370O',
 'DRD370P',
 'DRD370R',
 'DRD370T',
 'DRD370U',
 'DR1DAY_4.0',
 'DR1DAY_5.0',
 'DR1LANG_2.0',
 'DR1LANG_3.0',
 'DR1LANG_5.0',
 'DR1LANG_6.0',
 'DR1MNRSP_1.0',
 'DR1MNRSP_10.0',
 'DR1MNRSP_12.0',
 'DR1MNRSP_2.0',
 'DR1MNRSP_3.0',
 'DR1MNRSP_4.0',
 'DR1HELPD_11.0',
 'DR1HELPD_4.0',
 'DR1HELPD_5.0',
 'DR1HELPD_7.0',
 'DR1HELPD_9.0',
 'D

In [ ]:
X_train_lasso = X_train[features_names]
X_test_lasso = X_test[features_names]

In [ ]:
X_train_lasso

,RIDAGEYR,RIDRETH1,DMQMILIZ,DMDBORN4,DMDCITZN,SIAPROXY,AIALANGA,DMDHHSZA,DMDHHSZB,DMDHRAGE,...,INDHHIN2_3.0,INDHHIN2_8.0,INDHHIN2_nan,INDFMIN2_10.0,INDFMIN2_13.0,INDFMIN2_14.0,INDFMIN2_2.0,INDFMIN2_4.0,INDFMIN2_77.0,INDFMIN2_9.0
3780,0.078503,-0.072416,0.301172,-0.480544,-0.32286,0.753231,-0.286943,0.617551,0.777496,-0.834304,...,-0.254978,-0.287950,-0.103786,5.023955,-0.107706,-0.30073,-0.219262,-0.271581,-0.153211,-0.230908
3887,1.099997,0.718930,0.301172,-0.480544,-0.32286,0.753231,-0.286943,-0.644598,-0.878388,0.739235,...,-0.254978,-0.287950,-0.103786,-0.199046,9.284504,-0.30073,-0.219262,-0.271581,-0.153211,-0.230908
2015,-0.985553,-0.072416,0.301172,-0.480544,-0.32286,-1.327614,-0.286943,0.617551,0.777496,-1.293252,...,-0.254978,-0.287950,-0.103786,-0.199046,-0.107706,-0.30073,4.560757,-0.271581,-0.153211,-0.230908
1742,0.887186,0.718930,0.301172,-0.480544,-0.32286,0.753231,-0.286943,-0.644598,1.605438,0.411414,...,-0.254978,-0.287950,-0.103786,-0.199046,-0.107706,-0.30073,-0.219262,-0.271581,-0.153211,-0.230908
7196,-0.985553,0.718930,0.301172,-0.480544,-0.32286,-1.327614,-0.286943,0.617551,2.433380,0.673670,...,-0.254978,-0.287950,-0.103786,-0.199046,-0.107706,-0.30073,-0.219262,-0.271581,-0.153211,-0.230908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,-1.198364,-1.655107,0.301172,-0.480544,-0.32286,-1.327614,-0.286943,1.879701,-0.878388,1.460439,...,-0.254978,-0.287950,-0.103786,-0.199046,-0.107706,-0.30073,-0.219262,-0.271581,-0.153211,-0.230908
2333,-0.942990,0.718930,0.301172,-0.480544,-0.32286,-1.327614,-0.286943,-0.644598,2.433380,-0.375355,...,-0.254978,3.472825,-0.103786,-0.199046,-0.107706,-0.30073,-0.219262,-0.271581,-0.153211,-0.230908
8369,1.099997,0.718930,0.301172,-0.480544,-0.32286,0.753231,-0.286943,-0.644598,-0.878388,0.739235,...,-0.254978,-0.287950,-0.103786,-0.199046,-0.107706,-0.30073,-0.219262,-0.271581,-0.153211,-0.230908
3593,0.121065,0.718930,0.301172,-0.480544,-0.32286,0.753231,-0.286943,0.617551,-0.050446,-0.768739,...,-0.254978,3.472825,-0.103786,-0.199046,-0.107706,-0.30073,-0.219262,-0.271581,-0.153211,-0.230908


##SMOTE

In [ ]:
# Oversample and plot imbalanced dataset with SMOTE
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from matplotlib import pyplot
from numpy import where
# summarize class distribution
counter = Counter(y_train)
print(counter)
# transform the dataset
oversample = SMOTE()
undersample = RandomUnderSampler(random_state=0)
X_train_O, y_train_O = oversample.fit_resample(X_train_lasso, y_train)
X_train_U, y_train_U = undersample.fit_resample(X_train_lasso, y_train)

Counter({'carie': 1})


#TRADICIONAL

In [ ]:
#classificadores selecionados
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

In [ ]:
#OVERSAMPLE
logreg = LogisticRegression(random_state=0,max_iter=1000,verbose = False).fit(X_train_O, y_train_O)
svc = SVC(gamma='auto',verbose = False).fit(X_train_O, y_train_O)
arv = DecisionTreeClassifier(random_state=0).fit(X_train_O, y_train_O)
rfor = RandomForestClassifier(random_state=0).fit(X_train_O, y_train_O)
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train_O, y_train_O)
naive = BernoulliNB(force_alpha=True).fit(X_train_O, y_train_O)
#Predições
df_test =  pd.merge(X_test_lasso,y_test, left_index = True, right_index = True)
df_test = df_test.dropna(how='any')
X_test2 = df_test.drop(columns = ['carie'])
y_test = df_test[['carie']]
y_pred_logreg = logreg.predict(X_test2)
y_pred_svc = svc.predict(X_test2)
y_pred_arv = arv.predict(X_test2)
y_pred_rfor = rfor.predict(X_test2)
y_pred_knn = knn.predict(X_test2)
y_pred_naive = naive.predict(X_test2)

print("\nREGRESSÃO LOGÍSITCA\n")
print(classification_report(y_test, y_pred_logreg))
print("\nSVM\n")
print(classification_report(y_test, y_pred_svc))
print("\nÁRVORE DE DECISÃO\n")
print(classification_report(y_test, y_pred_arv))
print("\nRANDOM FOREST\n")
print(classification_report(y_test, y_pred_rfor))
print("\nKNN\n")
print(classification_report(y_test, y_pred_knn))
print("\nNAIVE-BAYES\n")
print(classification_report(y_test, y_pred_naive))



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-33-d160b8bd40d4>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfor = RandomForestClassifier(random_state=0).fit(X_train_O, y_train_O)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), 


REGRESSÃO LOGÍSITCA

              precision    recall  f1-score   support

           0       0.86      0.71      0.78       273
           1       0.74      0.87      0.80       254

    accuracy                           0.79       527
   macro avg       0.80      0.79      0.79       527
weighted avg       0.80      0.79      0.79       527


SVM

              precision    recall  f1-score   support

           0       0.84      0.71      0.77       273
           1       0.73      0.85      0.79       254

    accuracy                           0.78       527
   macro avg       0.78      0.78      0.78       527
weighted avg       0.79      0.78      0.78       527


ÁRVORE DE DECISÃO

              precision    recall  f1-score   support

           0       0.69      0.62      0.65       273
           1       0.63      0.70      0.66       254

    accuracy                           0.66       527
   macro avg       0.66      0.66      0.66       527
weighted avg       0.66   

In [ ]:
#UNDERSAMPLE
logreg = LogisticRegression(random_state=0,max_iter=1000,verbose = False).fit(X_train_U, y_train_U)
svc = SVC(gamma='auto',verbose = False).fit(X_train_U, y_train_U)
arv = DecisionTreeClassifier(random_state=0).fit(X_train_U, y_train_U)
rfor = RandomForestClassifier(random_state=0).fit(X_train_U, y_train_U)
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train_U, y_train_U)
naive = BernoulliNB(force_alpha=True).fit(X_train_U, y_train_U)
#Predições
df_test =  pd.merge(X_test_lasso,y_test, left_index = True, right_index = True)
df_test = df_test.dropna(how='any')
X_test2 = df_test.drop(columns = ['carie'])
y_test = df_test[['carie']]
y_pred_logreg = logreg.predict(X_test2)
y_pred_svc = svc.predict(X_test2)
y_pred_arv = arv.predict(X_test2)
y_pred_rfor = rfor.predict(X_test2)
y_pred_knn = knn.predict(X_test2)
y_pred_naive = naive.predict(X_test2)


print("\nREGRESSÃO LOGÍSITCA\n")
print(classification_report(y_test, y_pred_logreg))
print("\nSVM\n")
print(classification_report(y_test, y_pred_svc))
print("\nÁRVORE DE DECISÃO\n")
print(classification_report(y_test, y_pred_arv))
print("\nRANDOM FOREST\n")
print(classification_report(y_test, y_pred_rfor))
print("\nKNN\n")
print(classification_report(y_test, y_pred_knn))
print("\nNAIVE-BAYES\n")
print(classification_report(y_test, y_pred_naive))



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-34-f7d3290b4874>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfor = RandomForestClassifier(random_state=0).fit(X_train_U, y_train_U)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), 


REGRESSÃO LOGÍSITCA

              precision    recall  f1-score   support

           0       0.87      0.68      0.76       273
           1       0.72      0.89      0.80       254

    accuracy                           0.78       527
   macro avg       0.80      0.79      0.78       527
weighted avg       0.80      0.78      0.78       527


SVM

              precision    recall  f1-score   support

           0       0.87      0.62      0.72       273
           1       0.69      0.90      0.78       254

    accuracy                           0.75       527
   macro avg       0.78      0.76      0.75       527
weighted avg       0.78      0.75      0.75       527


ÁRVORE DE DECISÃO

              precision    recall  f1-score   support

           0       0.76      0.59      0.66       273
           1       0.64      0.80      0.71       254

    accuracy                           0.69       527
   macro avg       0.70      0.69      0.69       527
weighted avg       0.70   

##Tradicional sem SMOTE

In [ ]:
logreg = LogisticRegression(random_state=0,max_iter=1000,verbose = False).fit(X_train_lasso, y_train)
svc = SVC(gamma='auto',verbose = False).fit(X_train_lasso, y_train)
arv = DecisionTreeClassifier(random_state=0).fit(X_train_lasso, y_train)
rfor = RandomForestClassifier(random_state=0).fit(X_train_lasso, y_train)
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train_lasso, y_train)
naive = BernoulliNB(force_alpha=True).fit(X_train_lasso, y_train)
#Predições
df_test =  pd.merge(X_test_lasso,y_test, left_index = True, right_index = True)
df_test = df_test.dropna(how='any')
X_test2 = df_test.drop(columns = ['carie'])
y_test = df_test[['carie']]
y_pred_logreg = logreg.predict(X_test2)
y_pred_svc = svc.predict(X_test2)
y_pred_arv = arv.predict(X_test2)
y_pred_rfor = rfor.predict(X_test2)
y_pred_knn = knn.predict(X_test2)
y_pred_naive = naive.predict(X_test2)


print("\nREGRESSÃO LOGÍSITCA\n")
print(classification_report(y_test, y_pred_logreg))
print("\nSVM\n")
print(classification_report(y_test, y_pred_svc))
print("\nÁRVORE DE DECISÃO\n")
print(classification_report(y_test, y_pred_arv))
print("\nRANDOM FOREST\n")
print(classification_report(y_test, y_pred_rfor))
print("\nKNN\n")
print(classification_report(y_test, y_pred_knn))
print("\nNAIVE-BAYES\n")
print(classification_report(y_test, y_pred_naive))



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-35-f9033cf96282>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfor = RandomForestClassifier(random_state=0).fit(X_train_lasso, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,)


REGRESSÃO LOGÍSITCA

              precision    recall  f1-score   support

           0       0.80      0.80      0.80       273
           1       0.79      0.78      0.78       254

    accuracy                           0.79       527
   macro avg       0.79      0.79      0.79       527
weighted avg       0.79      0.79      0.79       527


SVM

              precision    recall  f1-score   support

           0       0.79      0.80      0.80       273
           1       0.78      0.78      0.78       254

    accuracy                           0.79       527
   macro avg       0.79      0.79      0.79       527
weighted avg       0.79      0.79      0.79       527


ÁRVORE DE DECISÃO

              precision    recall  f1-score   support

           0       0.67      0.69      0.68       273
           1       0.66      0.63      0.65       254

    accuracy                           0.66       527
   macro avg       0.66      0.66      0.66       527
weighted avg       0.66   

##SEM SMOTE E LASSO

In [ ]:
logreg = LogisticRegression(random_state=0,max_iter=1000,verbose = False).fit(X_train, y_train)
svc = SVC(gamma='auto',verbose = False).fit(X_train, y_train)
arv = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
rfor = RandomForestClassifier(random_state=0).fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
naive = BernoulliNB(force_alpha=True).fit(X_train, y_train)
#Predições
df_test =  pd.merge(X_test,y_test, left_index = True, right_index = True)
df_test = df_test.dropna(how='any')
X_test2 = df_test.drop(columns = ['carie'])
y_test = df_test[['carie']]
y_pred_logreg = logreg.predict(X_test2)
y_pred_svc = svc.predict(X_test2)
y_pred_arv = arv.predict(X_test2)
y_pred_rfor = rfor.predict(X_test2)
y_pred_knn = knn.predict(X_test2)
y_pred_naive = naive.predict(X_test2)


print("\nREGRESSÃO LOGÍSITCA\n")
print(classification_report(y_test, y_pred_logreg))
print("\nSVM\n")
print(classification_report(y_test, y_pred_svc))
print("\nÁRVORE DE DECISÃO\n")
print(classification_report(y_test, y_pred_arv))
print("\nRANDOM FOREST\n")
print(classification_report(y_test, y_pred_rfor))
print("\nKNN\n")
print(classification_report(y_test, y_pred_knn))
print("\nNAIVE-BAYES\n")
print(classification_report(y_test, y_pred_naive))


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-36-1a4cf61d55fd>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfor = RandomForestClassifier(random_state=0).fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for 


REGRESSÃO LOGÍSITCA

              precision    recall  f1-score   support

           0       0.78      0.79      0.78       128
           1       0.79      0.78      0.78       130

    accuracy                           0.78       258
   macro avg       0.78      0.78      0.78       258
weighted avg       0.78      0.78      0.78       258


SVM

              precision    recall  f1-score   support

           0       0.75      0.82      0.78       128
           1       0.81      0.73      0.77       130

    accuracy                           0.78       258
   macro avg       0.78      0.78      0.77       258
weighted avg       0.78      0.78      0.77       258


ÁRVORE DE DECISÃO

              precision    recall  f1-score   support

           0       0.67      0.66      0.66       128
           1       0.67      0.68      0.67       130

    accuracy                           0.67       258
   macro avg       0.67      0.67      0.67       258
weighted avg       0.67   

**MODELO ESCOLHIDO:** Devido ao melhor desempenho, o modelo escolhido foi o da regressão logística. Além de ter performado melhor, este é um dos mais simples e possui uma boa explicabilidade. A aplicação do SMOTE melhorou o desempenho do modelo, porém não de maneira significativa, diferente da regressão LASSO.

Tuning do Modelo Escolhido

In [ ]:
#identificar melhor função de ativação e melhor solver
parameters = {'C':(1,2.5,5,7.5,10),
              'solver':('lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga')}
logreg = LogisticRegression(random_state=0,max_iter=3000,verbose = False)
modelo = GridSearchCV(logreg, parameters, cv = 10)
modelo.fit(X_train_O, y_train_O)


In [ ]:
print(modelo.best_params_)

{'C': 2.5, 'solver': 'lbfgs'}
